In [1]:
import os
from dotenv import load_dotenv
import nest_asyncio

nest_asyncio.apply()
load_dotenv()

True

In [2]:
# Fill in the following values
PROVIDER = "github" # GitHub provider
MODEL = "vertex_ai/gemini-1.5-flash"

# Access the environment variables from the .env file
USER_TOKEN = os.environ.get('USER_TOKEN')  # GitHub user token - based on a each user account
PROJECT_ID = os.environ.get('PROJECT_ID')  # this is tied to your personal gcp's application_default_credentials.json

In [3]:
from pr_agent import cli
from pr_agent.config_loader import get_settings


def main(url: str, command: str = "review"):
    """
    Args:
        PR_URL (str): PR URL, for example 'https://github.com/Codium-ai/pr-agent/pull/809'
        COMMAND (str): Command to run (e.g. '/review', '/describe', '/ask="What is the purpose of this PR?"', ...). Defaults to "/review".
    """
    # Setting the configurations
    get_settings().set("CONFIG.git_provider", PROVIDER)
    get_settings().set("github.user_token", USER_TOKEN)

    # Set authentication    
    get_settings().set("vertexai.vertex_project", PROJECT_ID)
    get_settings().set("vertexai.vertex_location", None)
    
    # Set models
    get_settings().set("config.model", MODEL)
    get_settings().set("config.model_turbo", MODEL)
    get_settings().set("config.fallback_models",MODEL)

    # Run the command. Feedback will appear in GitHub PR comments
    cli.run_command(url, f"/{command}")

In [8]:
# PR URL, for example 'https://github.com/Codium-ai/pr-agent/pull/809'
PR_URL = "https://github.com/Q-Retail/qr_ngp_po_sub_routines/pull/636"

# Run pr-agent
# command = ['review', 'improve', 'describe', 'ask ...']
main(
    url=PR_URL,
    command="improve"
)

2024-09-16 09:45:58.398 | INFO     | pr_agent.tools.pr_code_suggestions:__init__:38 - Setting max_model_tokens to 14000 for PR improve
2024-09-16 09:45:58.881 | INFO     | pr_agent.tools.pr_code_suggestions:run:101 - Generating code suggestions for PR...
2024-09-16 09:46:05.085 | INFO     | pr_agent.tools.pr_code_suggestions:_prepare_prediction_extended:522 - Number of PR chunk calls: 1
2024-09-16 09:46:14.204 | INFO     | pr_agent.tools.pr_code_suggestions:_prepare_prediction_extended:547 - Removing suggestions 2 from call 0, because score is 0, and score_threshold is 1
